In [8]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.plugins.sparql import prepareQuery

# Load the ontology
g = Graph()
g.parse("FinalNewRdfXml.rdf", format="xml")

# Define the namespace
movie_proj = Namespace("http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project")

# Rule 1: Define a class for movies with at least one actor and one director
rule1 = (
    movie_proj[
    "MovieWithActorsAndDirectors"
],
    RDF.type,
    RDFS.Class
)
g.add(rule1)

# Rule 2: Define a class for movies where the actor and director are the same person
rule2 = (
    movie_proj[
    "SameActorAndDirector"
],
    RDF.type,
    RDFS.Class
)
g.add(rule2)

# Rule 3: Define a class for movies released after 2010
rule3 = (
    movie_proj[
    "RecentMovie"
],
    RDF.type,
    RDFS.Class
)
g.add(rule3)

# Save the updated RDF graph to the same file
g.serialize(destination="FinalNewRdfXml.rdf", format="xml")

# SPARQL query to check Rule 1: Movie names for movies with at least one actor and one director
query_movie_names = prepareQuery("""
    PREFIX movie_proj: <http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>

    SELECT DISTINCT ?movieName
    WHERE {
        ?movie a movie_proj:Movie .
        ?movie movie_proj:title ?movieName .
        ?movie movie_proj:hasActor ?actor .
        ?movie movie_proj:hasDirector ?director .
}
""", initNs={"movie_proj": movie_proj})

# Store unique movie names in a set
unique_movie_names = set()

# Extract and store movie names
for row in g.query(query_movie_names):
    movie_name = row.movieName.toPython() if isinstance(row.movieName, Literal) else row.movieName
    unique_movie_names.add(movie_name)

# Print the unique movie names
print("Rule 1: Movies with at least one actor and one director")
for movie_name in unique_movie_names:
    print(movie_name)

# SPARQL query to check Rule 2: Movies where the actor and director are the same person
query_actor_director_same_person = prepareQuery("""
    PREFIX movie_proj:<http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>

    SELECT ?movie ?title
    WHERE {
      ?movie a movie_proj:Movie ;
             movie_proj:hasActor ?person ;
             movie_proj:hasDirector ?person .
        OPTIONAL { ?movie movie_proj:title ?title
    }
}
""", initNs={"movie_proj": movie_proj})

# Print the movies where the actor and director are the same person
print("--------------------------------------------")
print("Rule 2: Movies where the actor and director are the same person")
for row in g.query(query_actor_director_same_person):
    print(f"{row.movie.split('/')[-1]}, Title: {row.title}")

# SPARQL query to check Rule 3: Recent Movies (Released after 2010)
query_rule3 = prepareQuery("""
    PREFIX movie_proj:<http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>

    SELECT ?movie ?title ?releaseYear
    WHERE {
      ?movie a movie_proj:Movie .
      ?movie movie_proj:year ?releaseYear .
      FILTER (?releaseYear > 2010)
      OPTIONAL { ?movie movie_proj:title ?title
    }
}
""", initNs={"movie_proj": movie_proj})

# Execute query for Rule 3
print("--------------------------------------------")
print("Rule 3: Recent Movies (Released after 2010)")
for row in g.query(query_rule3):
    print(f"Movie: {row.movie.split('/')[-1]}, Title: {row.title}, Release Year: {row.releaseYear}")

Rule 1: Movies with at least one actor and one director
Pulp Fiction
The Dark Knight
Parasite
FightClub
Snatch
Heat
Reservoir Dogs
Kill Bill (volume 1)
The Silence Of The Lambs
Life Is Beautiful
--------------------------------------------
Rule 2: Movies where the actor and director are the same person
Movie8, Title: Reservoir Dogs
Movie1, Title: Pulp Fiction
Movie10, Title: Life Is Beautiful
Movie9, Title: Parasite
Movie4, Title: FightClub
Movie2, Title: Kill Bill (volume 1)
--------------------------------------------
Rule 3: Recent Movies (Released after 2010)
Movie: Movie9, Title: Parasite, Release Year: 2019
